## Initial data solver using relaxation

We try to solve the time symmetric initial data when given a $ρ$ field on a cube. 
We put Robin boudary conditions.

Thus we shall be solving:

$$
ψ_{tt} = \Delta ψ + ψ^5 ρ \;\;\;\; \vec{x}\cdot \nabla ψ + (1-ψ) = 0
$$

$$
\rho = |\psi_t|^2 + V + \psi^{-4}|\nabla \psi|^2
$$

Using second order finite differences inside the domain and first order at the boudaries we get for the boundary values (which are not evolved):

For boundary conditions we shall put an outgoing wave, but taking into account that we want a radial wave and that something which at first order should be a monopolar field.

$$
ψ_t = (x^i\partial_i ψ - (ψ - ψ_0))/r
$$ 
Same for the time derivative, but that part should be unnessary becuse this should decay to a static solution.

The solution we are seeking is of the form of an outgoing wave plus a static solution, the one we are after. It well behave as a monopole at large distance, plus a constant.

$$
\psi(t,x) =\frac{f(t-r) + M}{r} + \psi_0
$$

Thus we have, 
$$
\partial_t \psi = \frac{f'(t-r)}{r}, \;\;\;\;\; \frac{\vec{x}\cdot \nabla \psi}{r} = -f'/r - f/r^2 - M/r^2 
$$

therefore asymtotically we should have, 
$$ 
\partial_t \psi = - \frac{\vec{x}\cdot \nabla \psi}{r} - (f+M)/r^2 = - \frac{\vec{x}\cdot ∇\psi}{r} - (\psi - \psi_0)/r 
$$

The equation for $\xi := \alpha \psi$ is:

$$
\xi_{tt} = \Delta \xi + \xi ψ^4 (\rho + 2S) \;\;\;\; \vec{x}\cdot \nabla ψ + (1-ψ) = 0
$$

Where

$$
S = - \psi^{-4}|\nabla \phi|^2 + 3(\pi^2 - V) \;\;\; \rho + 2S = 7\pi^2 - 5 V - \psi^{-4}|\nabla \phi|^2
$$

In [1]:
using SummationByPartsOperators
using Plots
using Revise
using Distributions
using FFTW
using LinearAlgebra
using Base.Threads
using HDF5
using FileIO
using JLD2
 
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")
#includet("../PIC-1D/aux_functions/aux_functions.jl")
includet("local_aux_functions.jl")



First run inidat.ipynb and get the data


In [2]:
#@show data_name = "data_3d_h_step_cut_medium_b" # failed with SF=4e-3 trying with 2e-3, failed, trying with 1e-3
#@show data_name = "data_3d_h_step_cut_small_bola" #Good with SF=4e-3, failed after T= 160 with F_full trying with the other.
@show data_name = "data_3d_step_cut_tiny_bola" #Good with SF=4e-3, failed after T= 160 with F_full trying with the other.
data = h5open("Inidats/" * data_name * ".h5","r")
@show enlarge_factor = 2 # or 3

data_name = "data_3d_step_cut_tiny_bola" = "data_3d_step_cut_tiny_bola"
enlarge_factor = 2 = 2


2

In [3]:
@show J_in = size(data["var6"])
@show Box_x_in = [read(data["coord0"])[1], read(data["coord0"])[end],read(data["coord1"])[1], read(data["coord1"])[end],read(data["coord2"])[1], read(data["coord2"])[end]];
@show dx_in = differentials(Box_x_in,J_in)
if enlarge_factor == 3 
    @show J = (3).*J_in
    @show Box_x = 3*Box_x_in;
elseif enlarge_factor == 2
    @show J = J_in .+ (2).*((J_in.÷2).+1)
    @show Box_x = [Box_x_in[1] - dx_in[1]*(J_in[1]÷2 +1),Box_x_in[2] + dx_in[1]*(J_in[1]÷2 +1)   , Box_x_in[3] - dx_in[2]*(J_in[2]÷2 +1), Box_x_in[4] + dx_in[2]*(J_in[2]÷2 +1) , Box_x_in[5] - dx_in[3]*(J_in[3]÷2 +1), Box_x_in[6] + dx_in[3]*(J_in[3]÷2 +1)];
end
dx = differentials(Box_x,J)
V = volume(Box_x)
m = 1
m2 = m^2


Dx = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
Dy = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
Dz = derivative_operator(MattssonNordström2004(), derivative_order = 1, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
D2x = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[1], xmax = Box_x[2], N=J[1])
D2y = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[3], xmax = Box_x[4], N=J[2])
D2z = derivative_operator(MattssonNordström2004(), derivative_order = 2, accuracy_order = 2, xmin = Box_x[5], xmax = Box_x[6], N=J[3])
x = collect(SummationByPartsOperators.grid(Dx))
y = collect(SummationByPartsOperators.grid(Dy))
z = collect(SummationByPartsOperators.grid(Dz));

J_in = size(data["var6"]) = (51, 51, 51)
Box_x_in = [(read(data["coord0"]))[1], (read(data["coord0"]))[end], (read(data["coord1"]))[1], (read(data["coord1"]))[end], (read(data["coord2"]))[1], (read(data["coord2"]))[end]] = [-25.0, 25.0, -25.0, 25.0, -25.0, 25.0]
dx_in = differentials(Box_x_in, J_in) = [0.9803921568627451, 0.9803921568627451, 0.9803921568627451]
J = J_in .+ 2 .* (J_in .÷ 2 .+ 1) = (103, 103, 103)
Box_x = [Box_x_in[1] - dx_in[1] * (J_in[1] ÷ 2 + 1), Box_x_in[2] + dx_in[1] * (J_in[1] ÷ 2 + 1), Box_x_in[3] - dx_in[2] * (J_in[2] ÷ 2 + 1), Box_x_in[4] + dx_in[2] * (J_in[2] ÷ 2 + 1), Box_x_in[5] - dx_in[3] * (J_in[3] ÷ 2 + 1), Box_x_in[6] + dx_in[3] * (J_in[3] ÷ 2 + 1)] = [-50.49019607843137, 50.49019607843137, -50.49019607843137, 50.49019607843137, -50.49019607843137, 50.49019607843137]


In [4]:
@show J; 
@show J_in; 
@show dx = differentials(Box_x,J);  
@show dx_in;
@show Box_x_in
@show Box_x;

J = (103, 103, 103)
J_in = (51, 51, 51)
dx = differentials(Box_x, J) = [0.980392156862745, 0.980392156862745, 0.980392156862745]
dx_in = [0.9803921568627451, 0.9803921568627451, 0.9803921568627451]
Box_x_in = [-25.0, 25.0, -25.0, 25.0, -25.0, 25.0]
Box_x = [-50.49019607843137, 50.49019607843137, -50.49019607843137, 50.49019607843137, -50.49019607843137, 50.49019607843137]


In [5]:
π2 = embed_source(read(data["var4"]),J, enlarge_factor);
#heatmap(x,y,π2[:,:,J[3]÷2], aspectratio=1)
∇2 = embed_source(read(data["var5"]),J, enlarge_factor);
#heatmap(x,y,∇2[:,:,J[3]÷2], aspectratio=1)
V = embed_source(read(data["var6"]),J, enlarge_factor);
#heatmap(x,y,V[:,:,J[3]÷2], aspectratio=1)

#heatmap(π2[:,:,(J[3]+1)÷2], aspectratio=1)

103×103×103 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [6]:
n_fields = 4 # the field and its derivative 
u = zeros(n_fields,J...);
du = similar(u);
d2 = zeros(2,J...);

In [7]:
#v = zeros(n_fields,data_steps+1 ,J...)
τ = 1.0 #damping must be positive here
par = (1.0, 1.0, τ) # (a, b, τ)
#par_F = (x,y,z,dxu,dyu,dzu,Dx,Dy,Dz,D2x,D2y,D2z,d2,dx,ρ,J, par)
dxu_x = zeros(2,n_fields,J[2],J[3]); dyu_x = copy(dxu_x); dzu_x = copy(dxu_x)
dxu_y = zeros(2,n_fields,J[1],J[3]); dyu_y = copy(dxu_y); dzu_y = copy(dxu_y)
dxu_z = zeros(2,n_fields,J[1],J[2]); dyu_z = copy(dxu_z); dzu_z = copy(dxu_z)
coors = (x,y,z)
boundary_derivs = (dxu_x,dyu_x,dzu_x,dxu_y,dyu_y,dzu_y,dxu_z,dyu_z,dzu_z)
derivs = (Dx,Dy,Dz,D2x,D2y,D2z)

source_factor = 0.004^2; SF = "4em3"
#source_factor = 0.001^2; SF = "1em3"
ρ = source_factor .*(π2,∇2,V)
par_FCR = (coors,boundary_derivs,derivs,d2,dx,ρ,J,n_fields,par)
#norms = zeros(n_fields,data_steps);



(([-50.49019607843137, -49.500192233756245, -48.510188389081115, -47.52018454440599, -46.53018069973087, -45.54017685505574, -44.55017301038062, -43.560169165705496, -42.570165321030366, -41.580161476355244  …  41.580161476355244, 42.570165321030366, 43.560169165705496, 44.55017301038062, 45.54017685505574, 46.53018069973087, 47.52018454440599, 48.510188389081115, 49.500192233756245, 50.49019607843137], [-50.49019607843137, -49.500192233756245, -48.510188389081115, -47.52018454440599, -46.53018069973087, -45.54017685505574, -44.55017301038062, -43.560169165705496, -42.570165321030366, -41.580161476355244  …  41.580161476355244, 42.570165321030366, 43.560169165705496, 44.55017301038062, 45.54017685505574, 46.53018069973087, 47.52018454440599, 48.510188389081115, 49.500192233756245, 50.49019607843137], [-50.49019607843137, -49.500192233756245, -48.510188389081115, -47.52018454440599, -46.53018069973087, -45.54017685505574, -44.55017301038062, -43.560169165705496, -42.570165321030366, -41

### Initial data

In [8]:
T = 0.0
t = 0.0
data_steps = 1
int_steps = 1
dt = 0.0

initial = true #false

if initial
    u[1,:,:,:] .= 1.0
    u[2,:,:,:] .= 0.0
    u[3,:,:,:] .= 1.0
    u[4,:,:,:] .= 0.0

else
    T_old = 0.0 
    data_steps_old = 1
    int_steps_old = 1
    dt_old = 0.0
    data_name_cont = "data_3d_h_step_cut_small_bola_F_full_T200"
    jldopen("Results/" * data_name_cont * ".jld2") do data
        par_evolv  = data["par_evolv"]
        global T_old, data_steps_old, int_steps_old, dt_old = par_evolv
        m = data_steps_old - 1
        tiempo = @sprintf("%05d", m)
        u[:,:,:,:] = data["u/u_$(tiempo)"]
    end
    @show T_old
    @show data_steps_old - 1
    t = T_old
    T = 2*T_old
    data_steps = data_steps_old
    int_steps = int_steps_old
end


@show norm(u)/prod(J)

@show size(u);
@show size(π2)
@show norm(π2)/sqrt(prod(J))


norm(u) / prod(J) = 0.0013528796394011995
size(u) = (4, 103, 103, 103)
size(π2) = (103, 103, 103)
norm(π2) / sqrt(prod(J)) = 0.010218250021095443


0.010218250021095443

In [ ]:


#run_name = data_name * "_FRC_full_T$(T)_SF$(SF)"
run_name = data_name * "_F_full_T$(T)_SF$(SF)"

if initial 
    t = 0
    T = 2400
    data_steps = 10 #place where to extract data
    int_steps = 1200
else
    run_name = run_name * "_cont"
end


full_outputs = true
file_name_out = "Results/" * run_name * ".jld2"#* ".h5"
run_pars = Dict("run_name" => run_name, "par_grid" => (Box_x, J,n_fields), "par_evolv" => (T,data_steps,int_steps,dt))
save(file_name_out, run_pars)

In [10]:
tiempo = @sprintf("%05d", 1)
field_name = "u"
file = jldopen(file_name_out, "a+") do file
    file[field_name * "/u_$(tiempo)"] = u
    file["norms/norms_$(tiempo)"] = get_norms(u)
end


4-element Vector{Float64}:
 1.0
 0.0
 1.0
 0.0

In [ ]:
prueba = false
if prueba
    t = 1.0
    T = 3.0
    data_steps = 2
    int_steps = 2
end

M = data_steps * int_steps # data steps is one less because we don't count the initial one.
@show dt = T/M
@show CFL = minimum(dt./dx)
t_series = zeros(data_steps+1)
t_series[1] = t

global t = 0.0
for m in 2:(data_steps+1)
    for mm in 1:int_steps
        #RK4_Step!(FCR_Full,u,t,dt,par_FCR)
        RK4_Step!(F_full,u,t,dt,par_FCR)
        global t = t + dt
    end    
    norms = get_norms(u)
    global t_series[m] = t_series[m-1] + int_steps * dt
    println("t = $(t_series[m]), m = $m, norm u_1 = $(norms[1]) norm u_2 = $(norms[2]), norm u_3 = $(norms[3]) norm u_4 = $(norms[4])")
    #v[:,m+1,:,:,:] = u[:,:,:,:]
    if full_outputs
        local tiempo = @sprintf("%05d", m)
        local field_name = "u"
        local file = jldopen(file_name_out, "a+") do file
            file[field_name * "/u_$(tiempo)"] = u
            file["norms/norms_$(tiempo)"] = norms
      end
    end
end

file = jldopen(file_name_out, "a+") do file
    file["t_series"] = t_series
end



dt = T / M = 2.0
CFL = minimum(dt ./ dx) = 2.018181818181818


InterruptException: InterruptException: